In [1]:
from pyrosetta import *
from pyrosetta.rosetta import *
from pyrosetta.teaching import *

from pyrosetta.rosetta.protocols.carbohydrates import *
from pyrosetta.rosetta.core.select.residue_selector import *
from pyrosetta.rosetta.core.simple_metrics.metrics import *
from pyrosetta.rosetta.core.simple_metrics.composite_metrics import *
from pyrosetta.rosetta.core.simple_metrics.per_residue_metrics import *

from scipy.spatial.transform import Rotation as R

options = """
-beta
-include_sugars
-alternate_3_letter_codes pdb_sugar

-write_pdb_link_records
-auto_detect_glycan_connections
-ignore_unrecognized_res
-out:level 100
"""

#-out:level 100

init(" ".join(options.split('\n')))

import os
import numpy as np
import pandas as pd
import copy

input_dir = "./"
os.chdir(input_dir)

PyRosetta-4 2021 [Rosetta PyRosetta4.Release.python38.mac 2021.36+release.57ac713a6e1d8ce6f60269b3988b1adac1d96fc6 2021-09-10T13:50:04] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


In [2]:
#Get list of proteins
pdb = [];

df = pd.read_csv('./carbbinders_pdblist.txt',header=None).values

print(len(df),df)

6559 [['3old']
 ['3ole']
 ['1pig']
 ...
 ['5DFM']
 ['4A34']
 ['5HQJ']]


In [3]:
def has_NXS(seq):
    """
    Gets whether a glycosylation NXS/T sequence exists
    Args:
        seq : pyrosetta pose sequence
    Returns:
        bool : True / False
    """
    for ii in range(len(seq)-2):
        if seq[ii] == "N":
            if seq[ii+2] == "S" or seq[ii+2] == "T":
                return True;
    return False
        
#Determines if the protein is glycosylated or we have free carbohydrates
def is_glycosylated(pose):
    """
    Gets whether a pose contains glycosylated proteins
    Args:
        pose : pyrosetta pose 
    Returns:
        bool : True / False
    """
    tree_set = pose.glycan_tree_set()
    for start in tree_set.get_start_points():
        parent = tree_set.get_parent(start)
        if parent != 0:
            return True;
    return False
    

def get_chain_seq(pose):
    """
    Gets pose sequence by cahin
    Args:
        pose : pyrosetta pose 
    Returns:
        chains (arr) : list of all sequences by internal ordering
    """
    chains = [];
    for ii in range(pose.num_chains()):
        r = pose.chain_begin(ii+1)
        c = pose.pdb_info().pose2pdb(r)
        c = c.split(' ')
        while '' in c:
            c.remove('')
        c = c[-1]
        chains.append(c)
    return chains
    
    
def get_carb_res(pose):
    """
    Args:
        pose : pyrosetta pose
    Returns:
        res (arr): contains [residue_number (pose_number),chain_number, carbo_dict]
            all carbohydrate residues with their associated parent chain and monomer species number
    """
    res = [];

    tree_set = pose.glycan_tree_set();
    if type(tree_set) == type(None):
        return res;
    
    for start in tree_set.get_start_points():
        chain = -1
        parent = tree_set.get_parent(start);
        if parent != 0:
            chain = pose.residue(parent).chain();
        curr_tree = tree_set.get_tree(start);
        for r in curr_tree.get_residues():
            name = pose.residue(r).carbohydrate_info().basic_name()
            try:
                res.append([r,chain,name ])
            except:
                res.append([r,chain,-1])
    return res

    


In [4]:

def get_protchainXYZ(pose,chain_num):

    """
    Args:
        pose : rosetta pose
        chain_num (int): chain number (rosetta numbering)
    Returns:
        p_coor (arr): array of protein coordinates
        p_label (arr): array with residue numbering (pose numbering)
    """
    
    p_coor = []; #protein coordinates
    p_label = []; # [RESIDUE_NUMBER] - rosetta numbering
    
    for jj in range(pose.chain_begin(chain_num),pose.chain_end(chain_num)+1):
        
        res_number = jj;
        num_of_atoms = pose.residue(res_number).natoms()
        for i in range(num_of_atoms):
            atom_name = pose.residue(res_number).atom_name(i+1).strip()
            if atom_name.count('H')> 0:
                continue
            if atom_name.startswith('V')> 0:
                continue
            curr = np.array(pose.residue(res_number).atom(i+1).xyz())
            p_coor.append(curr)
            p_label.append(res_number)

    return p_coor, p_label

def get_carbXYZ(pose,res):
    """
    Args:
        pose : rosetta pose
        res (arr): residues that the carbohydrates are
    Returns:
        coor (arr): array of protein coordinates
        label (arr): array with residue numbering (internal numbering)
    """
    
    coor = []; #protein coordinates
    label = []; # internal 'res' array numbering (0,1,2,3) NOT (245,246,555)
    
    for jj in range(len(res)):
        res_number = res[jj][0];
        num_of_atoms = pose.residue(res_number).natoms()
        for i in range(num_of_atoms):
            atom_name = pose.residue(res_number).atom_name(i+1).strip()
            if atom_name.count('H')> 0:
                continue
            if atom_name.startswith('V')> 0:
                continue
            curr = np.array(pose.residue(res_number).atom(i+1).xyz())
            coor.append(curr)
            label.append(jj)
    return coor, label

from scipy.spatial import distance_matrix as dm

def get_interact_residues(pose,prot,label,ligand,carb_label,carb_res,cutoff = 4.2):
    
    """
    Args:
        pose : rosetta pose
        prot (arr) : XYZ coor of all protein atoms
        label (arr): protein coordinate residue number labels
        ligand (arr) : XYZ coor of all carbohydrate atoms
        carb_label (arr): carbohydrate coordinate residue number labels
        carb_res (arr): carbohydrate labeling, in style of [ [residue_number (pose_number),chain_number, carbo_dict],...]
    Returns:
        res (arr): array of protein residues attached to carbohydrates
        label (arr): array with residue numbering (pose numbering)
    """
    
    if (len(ligand) == 0):
        return [];
    
    res = [];
    d = dm(prot,ligand)
    #return d
    #d = d < cutoff
    #print(np.sum(d))
    d = np.where(d < cutoff)
    #sz = np.shape(d)
    
    #go thru all interacting_residues
    for ii in range(len(d[0])):
        #get indices in relative terms
        i = d[0][ii]
        j = d[1][ii]
        prot_res = label[i]
        carb_r = carb_label[j]
        carb_chain = carb_res[carb_r][1]
        carb_mono = carb_res[carb_r][2]
        
        #check if parent chain of carb is same as prot_chain
        prot_chain = pose.residue(prot_res).chain()
        if carb_chain == prot_chain:
            continue;

        #create current label
        l = [prot_res, carb_mono]
            
        if (l not in res):
            res.append(l)
    return res;

def get_chain_coor(pose,chain):
    
    """
    function to get all CB and CA atom positions of all residues and their local frame of reference
    
    Args:
        pose : rosetta pose
        chain : rosetta pose chain number (1,2,...)
    Returns:
        cb (arr): array of all CB coordinates - glycine just CA
        ca (arr): array of all CA coordinates
        frame (arr) : array of all local frame ~
            x' = ca - n , y' = (ca - n) x (ca - c) , z' = x' x y'
    """
    
    start = pose.chain_begin(chain)
    end = pose.chain_end(chain)
    
    cb = np.zeros((end-start+1,3))
    ca = np.zeros((end-start+1,3))
    
    frame = np.zeros((end-start+1,3,3))
    
    for ii in range(start,end+1):
        res = pose.residue(ii);
        
        #get atom coordinates
        xyz = res.xyz('N')
        n = np.array([xyz[0],xyz[1],xyz[2]])
        xyz = res.xyz('CA')
        a = np.array([xyz[0],xyz[1],xyz[2]])
        xyz = res.xyz('C')
        c = np.array([xyz[0],xyz[1],xyz[2]])
        b = a
        
        name = res.name1();
        if name != "G":
            xyz = res.xyz('CB')
            b = np.array([xyz[0],xyz[1],xyz[2]])
            
        #get reference frame
        ca_n = a - n;
        ca_n /= np.linalg.norm(ca_n);
        x_prime = ca_n;
        ca_c = a - c;
        ca_c /= np.linalg.norm(ca_c);
        y_prime = np.cross(ca_n,ca_c);
        y_prime /= np.linalg.norm(y_prime);
        z_prime = np.cross(x_prime,y_prime);
        z_prime /= np.linalg.norm(z_prime);
        
        #explcitly define as
        #        [ -x'- ]
        # ref =  [ -y'- ]
        #        [ -z'- ]
        ref = np.zeros((3,3));
        ref[0,:] = x_prime;
        ref[1,:] = y_prime;
        ref[2,:] = z_prime;
        
        #update
        cb[ii-start,:] = b;
        ca[ii-start,:] = a;
        frame[ii-start,...] = ref
        
    return cb, ca, frame

In [5]:
os.getcwd()

'/Users/scanner1/Downloads/capdock'

In [6]:
pose = pyrosetta.toolbox.pose_from_rcsb('1bag',ATOM=False)

In [7]:
carbs = get_carb_res(pose)
print(carbs)

[[429, -1, 'glucose'], [430, -1, 'glucose'], [431, -1, 'glucose'], [432, -1, 'glucose'], [433, -1, 'glucose']]


In [8]:
tree_set = pose.glycan_tree_set()

In [9]:
s = []
for start in tree_set.get_start_points():
    print(start, pose.pdb_info().pose2pdb(start), pose.residue_type(start).name3(), pose.residue_type(start).name())
    s.append(start)

429 1 B  Glc ->4)-beta-D-Glcp:reducing_end


In [10]:
my_res = []
for ii in s:
    for jj in carbs:
        if jj[0] == ii:
            if jj[1] == -1:
                my_res.append(ii)

In [11]:
print(tree_set.n_trees())

1


In [12]:
tree1 = tree_set.get_tree(my_res[0])

In [13]:
carb_full_name = ""
for res in tree1.get_residues():
    #print(res, pose.residue_type(res).name3(), pose.residue_type(res).name())
    carb_full_name += pose.residue_type(res).base_name()

In [14]:
carb_full_name

'->4)-beta-D-Glcp->4)-alpha-D-Glcp->4)-alpha-D-Glcp->4)-alpha-D-Glcp->3)-alpha-D-Glcp'

In [15]:
from pyrosetta.rosetta.core.pose import pose_from_saccharide_sequence

In [16]:
#s = '->4)-beta-D-Glcp->4)-alpha-D-Glcp->4)-alpha-D-Glcp->4)-alpha-D-Glcp->3)-alpha-D-Glcp'

carb = pose_from_saccharide_sequence(carb_full_name)

s = '->4)-beta-D-Fucp->4)-alpha-D-Fucp->4)-alpha-D-Manp->4)-alpha-D-ManpNAc->3)-alpha-D-Fucf'
carb = pose_from_saccharide_sequence(s)

In [17]:
carb.sequence()

'ZZZZZ'

In [18]:
for i in range(1,carb.size()):
    r = carb.residue(i)
    #print(r.all_bb_atoms())
    #r.carbohydrate_info()
    print(r,'\n')
#print(r.xyz(1))

#r.ring_atoms

Residue 1: ->3)-alpha-D-Fucf:reducing_end (Fuc, Z):
Base: ->3)-alpha-D-Fucf
 Properties: POLYMER CARBOHYDRATE LOWER_TERMINUS POLAR CYCLIC HEXOSE ALDOSE D_SUGAR FURANOSE ALPHA_SUGAR C6_MODIFIED DEOXY_SUGAR
 Variant types: LOWER_TERMINUS_VARIANT C6_DEOXY_SUGAR
 Main-chain atoms:  C1   C2   C3   O3 
 Backbone atoms:    C1   C2   C3   O3   C4   O4   VO4  VC1  H1   H2   H3   H4 
 Ring atoms:    C1   C2   C3   C4   O4 
 Side-chain atoms:  O1   O2   C5   O5   C6   O6   HO3  HO6  HO1  HO2  H5   HO5 1H6  2H6  3H6 
Carbohydrate Properties for this Residue:
 Basic Name: fucose
 IUPAC Name: ->3)-alpha-D-fucofuranose
 Abbreviation: ->3)-alpha-D-Fucf
 Classification: aldohexose
 Stereochemistry: D
 Ring Form: furanose
 Anomeric Form: alpha
 Modifications: 
  deoxy sugar
 Polymeric Information:
  Reducing?: yes
  Main chain connection: (_->3)
  Branch connections: none
Ring Conformer: 2T3 (twist): C-P parameters (q, phi, theta): 0.4, 0; nu angles (degrees): 54, -64, 54, -22, -22
  O1 : axial
  O2 : e

In [19]:


out_fasta = "";
bad = [];

print("Hello")

for ii in range(len(ls)):
    has_carb = True;

    f = '../preprocess/capsif_pdb/' + ls[ii]
    #only run for single pdb
        
    try:
        #signal.signal(signal.SIGALRM, timeout_handler)
        #signal.alarm(1)
        pose = [];
        
        #try:
        pose = get_pose(f)
        #except:
        #    print("Pose took over a minute to load")
        #    continue;
        
        chains = get_chain_seq(pose)
        
        #First and foresmost - find the carbohydrate
        
        carb_res = get_carb_res(pose);
        if (len(carb_res) == 0):
            #print("Carb not detected")
            has_carb = False;
        #break;

        ag_coor, ag_label = get_carbXYZ(pose,carb_res)
        
        print(ii,len(ls),ls[ii],has_carb)
        
        #go thru all protein chains
        nc = pose.num_chains();
        for c in range(1,nc+1):
            
            #only protein chains allowed
            if pose.residue(pose.chain_begin(c)).is_protein() == False:
                continue;
            
            coor, label = get_protchainXYZ(pose,c)
            cb, ca, frame = get_chain_coor(pose,c)
            
            #ßprint(len(ag_coor))
            #get interacting residues
            res = get_interact_residues(pose,coor,label,ag_coor,ag_label,carb_res)
            res = np.array(res) # for readability
            #print(res)

            seq = pose.chain_sequence(c)
            out_fasta += ">" + ls[ii][:4] + "_" + str(c) + "\n"
            out_fasta += seq + "\n"

            #get the label, recall in pose numbering
            new_label = np.zeros(( pose.chain_end(c) - pose.chain_begin(c) + 1 , len(carb_dict) + 1 ) )
            for r in range(pose.chain_begin(c), pose.chain_end(c) ):
                #if no protein binds it
                if (len(res) == 0):
                    out_fasta += "x,"
                    continue;

                if r in res[:,0]:
                    new_label[r-pose.chain_begin(c),0] = 1;
                    
                    ind = np.where(res[:,0] == r)[0]
                    #print(ind)
                    out_fasta += str(res[ind[0],1])
                    new_label[r-pose.chain_begin(c),res[ind[0],1]+1] = 1
                    for kk in range(1,len(ind)):
                        out_fasta += "|" + str(res[kk,1])
                        new_label[r-pose.chain_begin(c),res[kk,1]+1] = 1
                else:
                    out_fasta += "x"
                out_fasta += ","
            out_fasta += "\n"

            #os.path.abspath(file_seq)
            f_seq = open(out_file,'w+')
            f_seq.write(out_fasta)
            f_seq.close()
            #print(a_seq,'\n',b_seq,'\n',ag_seq)
            
            np.savez('../preprocess/capsif_coor/' + ls[ii][:4] + "_" + str(c) + ".npz",ca=ca,cb=cb,frame=frame,label=new_label)

    except:
        bad.append(ls[ii])
    #break;
    
    


Hello
0 808 3R4Z_1.pdb True
1 808 5A6M_1.pdb True
2 808 5GOQ_1.pdb True
3 808 4RK0_1.pdb True
4 808 6ROE_1.pdb True
5 808 3PUN_1.pdb True
6 808 1X9D_1.pdb True
7 808 4WZT_1.pdb True
8 808 3KZH_1.pdb True
9 808 2WMG_1.pdb True
10 808 1SZ2_1.pdb True
11 808 4XFE_1.pdb True
12 808 4XR8_2.pdb True
13 808 2E9M_1.pdb True
14 808 1A3K_1.pdb True
15 808 2DTX_1.pdb True
16 808 4QQQ_1.pdb True
17 808 3RSJ_1.pdb True
18 808 4LO2_1.pdb True
19 808 3VV1_1.pdb True
20 808 6UAS_1.pdb True
21 808 2CGY_1.pdb True
22 808 1K1W_1.pdb True
23 808 2J44_1.pdb True
24 808 3QSP_1.pdb True
25 808 4NFD_1.pdb True
26 808 4UA8_1.pdb True
27 808 5U3E_1.pdb True
28 808 1SL6_1.pdb True
29 808 6R2J_1.pdb True
30 808 4TF4_1.pdb True
31 808 4YFZ_1.pdb True
32 808 5MQS_1.pdb True
33 808 4XRE_1.pdb True
34 808 1RYD_1.pdb True
35 808 4B8S_1.pdb True
36 808 5KGA_1.pdb True
37 808 5GTG_1.pdb True
38 808 3M3Q_1.pdb True
39 808 3OO6_1.pdb True
40 808 5DEQ_1.pdb True
41 808 3SEJ_1.pdb True
42 808 4YW2_1.pdb True
43 808 6ER3_1.p

In [20]:
print(bad)
print(out_fasta)

['3OSQ_1.pdb', '3OSR_1.pdb']
>3R4Z_1
KLSKASLRAIERGYDEKGPEWLFEFDITPLKGDLAYEEGVIRRDPSAVLKVDDEYHVWYTKGEGETVGFGSDNPEDKVFPWDKTEVWHATSKDKITWKEIGPAIQRGAAGAYDDRAVFTPEVLRHNGTYYLVYQTVKAPYLNRSLEHIAIAYSDSPFGPWTKSDAPILSPENDGVWDTDEDNRFLVKEKGSFDSHKVHDPCLMFFNNRFYLYYKGETMGESMNMGGREIKHGVAIADSPLGPYTKSEYNPITNSGHEVAVWPYKGGMATMLTTDGPEKNTCQWAEDGINFDIMSHIKGAPEAVGFFRPDDPISGIEWGLSHKYDASWNWNYLCFFKTRRQVLDAGSYQQTGDSGAVHH
x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,4,x,4,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,4,x,4,x,x,x,x,x,x,x,x,x,x,4,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,4,4,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,

In [65]:
type(carb_res)

NoneType

In [66]:
type(carb_res) == type(None)

True

In [48]:
pose.num_chains()

2

In [50]:
pose.chain_begin(1)
pose.residue(1).is_protein()

True

In [27]:
#print(a_coor)
a_label
carb_res

[[871, -1, 10], [872, -1, 10], [873, -1, 10]]

In [37]:
print(np.shape(a_coor))

(1643, 3)


In [70]:
#res = np.array(res)